In [1]:
import pandas as pd
df= pd.read_excel("C:\\lrdata.xlsx")

In [10]:
df1=df.drop(['percentage_salary_hike','years_with_curr_manager','dummy_sales','years_in_current_role','training_times_last_year','dummy_single','dummy_yes','job_level','years_at_company','monthly_income','dummy_female','dummy_divorced','dummy_HR','age','total_working_years'],axis=1)

In [11]:
import statsmodels.api as sm
X=df1
Y=df['percentage_salary_hike']
X = sm.add_constant(X)
model1=sm.OLS(Y,X)
result=model1.fit()
print(result.summary())

                              OLS Regression Results                              
Dep. Variable:     percentage_salary_hike   R-squared:                       0.600
Model:                                OLS   Adj. R-squared:                  0.599
Method:                     Least Squares   F-statistic:                     1099.
Date:                    Sat, 25 Jan 2020   Prob (F-statistic):          2.34e-292
Time:                            20:59:29   Log-Likelihood:                -3319.7
No. Observations:                    1470   AIC:                             6645.
Df Residuals:                        1467   BIC:                             6661.
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [20]:
df1=df.drop(['percentage_salary_hike'],axis=1)
X=df1
Y=df['percentage_salary_hike']

In [21]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


In [23]:
result = stepwise_selection(X, Y)

print('resulting features:')
print(result)

Add  performance_rating             with p-value 4.2716e-293
Add  years_at_company               with p-value 0.0194073
resulting features:
['performance_rating', 'years_at_company']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
